<h1><center> Gesture detection and classification to detect confusion using MediaPipe, Locally Linear Embedding and KMeans clustering. </center></h1>

## Importing the required libraries

In [1]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import glob
import math
import numpy as np
import seaborn as sns
sns.set()
%matplotlib inline
%matplotlib widget
# %matplotlib notebook

<h1><center> Step 1: Pre-processing </center></h1>

## Part 1: Function to get joints' coodinates from an image 

In [2]:
def get_ten_joints(image):
#     image = cv2.imread(image)
    image_vector = []
    mp_pose = mp.solutions.pose
    # Initialize MediaPipe Pose.
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
#     print(pose)
    mp_drawing = mp.solutions.drawing_utils 
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#     results = pose.process(image)
#     print('results = ', results.pose_landmarks)
    # Print nose landmark.
    image_hight, image_width, image_depth = image.shape
    #     if not results.pose_landmarks:
    #         continue
#     print(results.pose_landmarks)
# asjdnaisdbasdhbhkasbkahdbasidhan akjnfbqiu
    if(results.pose_landmarks is None ):
        return ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
    left_shoulder = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width, 
                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_hight)
    #     print("Left Shoulder Location : ", left_shoulder)
    right_shoulder = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width,
                  results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_hight)
    #     print("Right Shoulder Location : ", right_shoulder)
    nose_cood = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width,
                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_hight,
                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z * image_depth)
    
    #     print("Nose Location : ", nose_cood)

    # Creating a centroid (point between nose and shoulders as the new origin (0,0))
    
    centroid = ((left_shoulder[0] + right_shoulder[0] + nose_cood[0]) / 3 , (left_shoulder[1] + right_shoulder[1] + nose_cood[1]) / 3)
    #     print("Centroid = ", centroid)
    
    centroid_point = (int(centroid[0]), int(centroid[1]))
    cv2.circle(image, centroid_point, 3, (255,0,0), -1)
    
    # Calculating the point for head/face and replacing the values in the landmakrs 
    
    left_ear = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x * image_width, 
               results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y * image_hight)
    right_ear = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x * image_width, 
                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y * image_hight)
# Now we have to replace the values from loc 0(nose) to loc 10 (right mouth) and put a general point for head which is mid
# point of 2 ears
    
    head = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
#     print("Head = ", head)
    head_point = (int(head[0]), int(head[1]))
    cv2.circle(image, head_point, 3, (0,0,255), -1)
    
# Updating removing the 10 points and putting 1 head point 
    allJointsXarray = []
    allJointsYarray = []
    
    for i in mp_pose.PoseLandmark:
        allJointsXarray.append((results.pose_landmarks.landmark[i].x) * image_width)
        allJointsYarray.append((results.pose_landmarks.landmark[i].y) * image_hight)
    
    tenJointsXArray = allJointsXarray[11:25]
    tenJointsYArray = allJointsYarray[11:25]
    tenJointsXArray.insert(0, head[0])
    tenJointsYArray.insert(0, head[1])
    tenJointsXArray.insert(1, centroid[0])
    tenJointsYArray.insert(1, centroid[1])

# Removing the unnecessary points
    for i in range(6):
        del tenJointsXArray[6]
    for i in range(6):
        del tenJointsYArray[6]
    
# Redifining points wrt to new centroid   
    xCoordinatesCentroid = list(map(lambda x: x - centroid[0], tenJointsXArray))
    yCoordinatesCentroid = list(map(lambda y: y - centroid[1], tenJointsYArray))
    

    
    
    current_vector = []
    for i in range(len(xCoordinatesCentroid)):
        current_vector.append((xCoordinatesCentroid[i], yCoordinatesCentroid[i]))
    image_vector.append(current_vector)
    
    
    
    
    
    
# Drawing a rectangle around the min and max point of the image. 
    rectBottomLeft = (math.floor(min(tenJointsXArray)), math.floor(min(tenJointsYArray)))
    rectTopRight = (math.ceil(max(tenJointsXArray)), math.ceil(max(tenJointsYArray)))
   
    

    cv2.circle(image, rectBottomLeft, 3, (0,0,255), -1)
    cv2.circle(image, rectTopRight, 3, (0,0,255), -1)
    cv2.rectangle(image, rectBottomLeft, rectTopRight, (255,0,0), 1)
    
# Draw pose landmarks.
#     print(f'Pose landmarks of {name}:')
#     annotated_image = image.copy()
#     mp_drawing.draw_landmarks(
#       image=annotated_image,
#       landmark_list=results.pose_landmarks,
#       connections=mp_pose.POSE_CONNECTIONS,
#       landmark_drawing_spec=drawing_spec,
#       connection_drawing_spec=drawing_spec)
#     cv2.imshow('annotated_iamge', annotated_image)
#     plt.figure()
#     plt.imshow(annotated_image)
#     cv2.waitKey(0)
#     newXArray = list(map(lambda x: x - centroid[0], xArray))
#     newYArray = list(map(lambda y: y - centroid[1], yArray))
#     head_plotX = [newXArray[0], newXArray[1]]
#     head_plotY = [newYArray[0], newYArray[1]]
#     hands_plotX = [newXArray[6], newXArray[4], newXArray[2],newXArray[3],newXArray[5],newXArray[7]]
#     hands_plotY = [newYArray[6], newYArray[4], newYArray[2],newYArray[3],newYArray[5],newYArray[7]]
#     body_plotX = [newXArray[2], newXArray[8], newXArray[9], newXArray[3]]
#     body_plotY = [newYArray[2], newYArray[8], newYArray[9], newYArray[3]]
#     plt.plot(head_plotX, head_plotY, 'o', color='black' , linewidth=1, linestyle='solid')
#     plt.plot(hands_plotX, hands_plotY, 'o', color='black', linewidth=1, linestyle='solid')
#     plt.plot(body_plotX, body_plotY, 'o', color='black', linewidth=1, linestyle='solid')
#     plt.yscale("linear")
#     plt.ylim(max(newYArray) + 20, min(newYArray) - 20)
#     plt.xlim(-300, 300)
#     plt.show()
#     plt.imshow(annotated_image)
   
    final = []
    for i in image_vector:
        height = int(abs(i[-2][1] - i[2][1]))
        width = int(abs(i[3][0] - i[2][0]))
#         print(height, width)

        y = 150 / height
        x = 50 / width

        ratio = (y, x)
        temp = list()
        for j in i:
            temp.append((j[0] * x, j[1] * y))
        final.append(temp)
    
    normalizedXarray = list()
    normalizedYarray = list()
    
    for z in final:
        xList = []
        yList = []
        for x, y in z:
            xList.append(x)
            yList.append(y)
        normalizedXarray = xList
        normalizedYarray = yList
    
#     plt.figure(figsize=(10,10))
    newXArray = normalizedXarray
    newYArray = normalizedYarray
#     for i in range(len(image_vector)):
#         head_plotX = [normalizedXarray[0], normalizedXarray[1]]
#         head_plotY = [normalizedYarray[0], normalizedYarray[1]]
#         hands_plotX = [normalizedXarray[6], normalizedXarray[4], normalizedXarray[2],normalizedXarray[3],normalizedXarray[5],normalizedXarray[7]]
#         hands_plotY = [normalizedYarray[6], normalizedYarray[4], normalizedYarray[2],normalizedYarray[3],normalizedYarray[5],normalizedYarray[7]]
#         body_plotX = [normalizedXarray[2], normalizedXarray[8], normalizedXarray[9], normalizedXarray[3]]
#         body_plotY = [normalizedYarray[2], normalizedYarray[8], normalizedYarray[9], normalizedYarray[3]]
#         plt.plot(head_plotX, head_plotY, 'o', color='black', linewidth=1, linestyle='solid', label=i)
#         plt.plot(hands_plotX, hands_plotY, 'o', color='black', linewidth=1, linestyle='solid')
#         plt.plot(body_plotX, body_plotY, 'o', color='black', linewidth=1, linestyle='solid')
#     plt.gca().invert_yaxis()
#     plt.legend(loc='best')
#     plt.show()
    
    return (normalizedXarray, normalizedYarray)

### Testing the get_ten_joints function

In [3]:
images = {name: cv2.imread(name) for name in glob.glob('./Images\confused/mixedsamples/*jpg')}

for name, image in images.items():
#     plt.figure()
#     plt.imshow(image)
    print(name)   
#     cv2.imshow('image',image)
#     cv2.waitKey(0)
    normalized_X_coordinates, normalized_Y_coordinates = get_ten_joints(image)
    print("normalized_X_coordinates = ", normalized_X_coordinates)
    print("normalized_Y_coordinates = ", normalized_Y_coordinates)

./Images\confused/mixedsamples\chin (1).jpg
normalized_X_coordinates =  [-3.359994141662374, 0.0, 28.26364443023154, -21.80330915019867, 39.93959417368503, -23.860618123348722, -5.954259737057889, -5.906477855875137, 23.32287419666635, -11.214934257750816]
normalized_Y_coordinates =  [-52.12860262912253, 0.0, 24.581267664083846, 27.48169328855431, 133.94079542913926, 128.88124455576357, 101.65123953649649, 9.987417888264405, 175.15440349993497, 176.840413936042]
./Images\confused/mixedsamples\chin (2).jpg
normalized_X_coordinates =  [-1.6482894308865073, 0.0, 25.478429091162983, -24.966619908809662, 30.248492606915537, -24.946913495659828, -12.149062869139016, -7.343390840105713, 14.65752145741135, -17.035572067834437]
normalized_Y_coordinates =  [-49.350560162247724, 0.0, 20.58595529123515, 19.555829853570767, 126.79196325670772, 115.24146845360764, 102.44679425944801, -5.459656084285057, 170.6611491051041, 167.71936086045596]
./Images\confused/mixedsamples\chin (3).jpg
normalized_X_c

normalized_X_coordinates =  [-0.9889931869095386, 0.0, 25.047392778283434, -25.073433128118264, 40.58213630026083, -40.20452242195928, 34.94448286301366, -35.45481693140255, 17.42056606914104, -12.859204569037813]
normalized_Y_coordinates =  [-43.29092860221863, 0.0, 15.769205689430237, 20.76162278652191, 88.32712233066559, 97.0665317773819, 170.41314661502838, 169.11242544651031, 166.1062377691269, 169.17374074459076]
./Images\confused/mixedsamples\frame6300.jpg
normalized_X_coordinates =  [-3.4264978805890802, 0.0, 26.16481115384797, -23.92412637832219, 50.92674099294432, -38.14506301237214, 40.20421915249418, -38.587609209595456, 19.530005977185347, -14.181086589569931]
normalized_Y_coordinates =  [-45.459820378211234, 0.0, 13.94166100409723, 21.09256867439516, 103.55596003993864, 120.02782091017691, 33.5582475508413, 44.18389643392255, 164.04693741952218, 164.04223057531541]
./Images\confused/mixedsamples\frame6450.jpg
normalized_X_coordinates =  [-4.147227548619607, 0.0, 27.450106

## Part 2: Function to get frames from a video on which we will use the created fucntion to obtain the coordinates of the joints 

In [4]:
# import cv2
# import os
# cap = cv2.VideoCapture('sampleOP.mp4')
# count = 0
# path = './Images\sampleOP'
# fps = cap.get(cv2.CAP_PROP_FPS)
# print(fps)
# while cap.isOpened():
#     ret, frame = cap.read()

#     if ret:
#         cv2.imwrite(os.path.join(path, 'frame{:d}.jpg'.format(count)), frame)
#         count += 150 # i.e. at 30 fps, this advances one second
#         cap.set(1, count)
#     else:
#         cap.release()
#         break

### We store the images at above 'path' and we run the function for each image 

In [5]:
images = {name: cv2.imread(name) for name in glob.glob('./Images\confused/mixedsamples/*jpg')}
LLEVideoArray = []
for name, image in images.items():
#     plt.figure()
#     plt.imshow(image)
    print(name)   
#     cv2.imshow('image',image)
#     cv2.waitKey(0)s
    normalized_X_coordinates, normalized_Y_coordinates = get_ten_joints(image)
    print("normalized_X_coordinates = ", normalized_X_coordinates)
    print("normalized_Y_coordinates = ", normalized_Y_coordinates)
    LLEVideoArray.append([normalized_X_coordinates, normalized_Y_coordinates])

./Images\confused/mixedsamples\chin (1).jpg
normalized_X_coordinates =  [-3.359994141662374, 0.0, 28.26364443023154, -21.80330915019867, 39.93959417368503, -23.860618123348722, -5.954259737057889, -5.906477855875137, 23.32287419666635, -11.214934257750816]
normalized_Y_coordinates =  [-52.12860262912253, 0.0, 24.581267664083846, 27.48169328855431, 133.94079542913926, 128.88124455576357, 101.65123953649649, 9.987417888264405, 175.15440349993497, 176.840413936042]
./Images\confused/mixedsamples\chin (2).jpg
normalized_X_coordinates =  [-1.6482894308865073, 0.0, 25.478429091162983, -24.966619908809662, 30.248492606915537, -24.946913495659828, -12.149062869139016, -7.343390840105713, 14.65752145741135, -17.035572067834437]
normalized_Y_coordinates =  [-49.350560162247724, 0.0, 20.58595529123515, 19.555829853570767, 126.79196325670772, 115.24146845360764, 102.44679425944801, -5.459656084285057, 170.6611491051041, 167.71936086045596]
./Images\confused/mixedsamples\chin (3).jpg
normalized_X_c

normalized_X_coordinates =  [-2.6603746244491635, 0.0, 25.918532945083133, -24.10729839284225, 43.79478807551157, -43.75133989544526, 82.5307038330947, -58.255898146442675, 16.194896765875225, -16.510844654884202]
normalized_Y_coordinates =  [-47.3774615235216, 0.0, 20.45503000574788, 19.872344384981893, 105.33303163183018, 100.13767302505615, 109.04064704114059, 28.095954985130493, 170.84496040043868, 170.0367664727639]
./Images\confused/mixedsamples\frame5250.jpg
normalized_X_coordinates =  [-2.153852324941504, 0.0, 25.925007177677458, -24.214800143297463, 45.944991923156586, -39.50013869848007, 64.1829528452911, -61.46939373238659, 21.392346539975318, -12.148959375483734]
normalized_Y_coordinates =  [-46.60391533861355, 0.0, 18.567843704807515, 25.297270745647197, 98.54353264886505, 113.64363590065314, 65.38399020019843, 66.15365828786577, 168.5802577709665, 171.02130882594048]
./Images\confused/mixedsamples\frame5850.jpg
normalized_X_coordinates =  [-0.9889931869095386, 0.0, 25.047

<h1><center> Step 2: Generate an input for LLE model 

In [6]:
LLEFrames = []
for k in range(len(LLEVideoArray)):
    temp = []
    for i in range(len(LLEVideoArray[k][0])):
        temp.append(LLEVideoArray[k][0][i])
        temp.append(LLEVideoArray[k][1][i])           
    LLEFrames.append(temp)
print(LLEFrames)

[[-3.359994141662374, -52.12860262912253, 0.0, 0.0, 28.26364443023154, 24.581267664083846, -21.80330915019867, 27.48169328855431, 39.93959417368503, 133.94079542913926, -23.860618123348722, 128.88124455576357, -5.954259737057889, 101.65123953649649, -5.906477855875137, 9.987417888264405, 23.32287419666635, 175.15440349993497, -11.214934257750816, 176.840413936042], [-1.6482894308865073, -49.350560162247724, 0.0, 0.0, 25.478429091162983, 20.58595529123515, -24.966619908809662, 19.555829853570767, 30.248492606915537, 126.79196325670772, -24.946913495659828, 115.24146845360764, -12.149062869139016, 102.44679425944801, -7.343390840105713, -5.459656084285057, 14.65752145741135, 170.6611491051041, -17.035572067834437, 167.71936086045596], [-0.24794960158995483, -46.313237457375266, 0.0, 0.0, 25.125425094845646, 20.338835891004628, -25.1131344800708, 19.422016780413877, 35.09058897522674, 120.64250907348713, -24.164784811008943, 110.4815274633038, -1.098772712137514, 119.7927418179537, 1.5031

<h1><center> Step 3: Using the LLE model to see the difference between gestures 

In [7]:
from sklearn import manifold, datasets
LLEReduced, err = manifold.locally_linear_embedding(LLEFrames, n_neighbors=20, n_components=2)
print("Done. Reconstruction error: %g" % err)
images = {name: cv2.imread(name) for name in glob.glob('./Images\confused/mixedsamples/*jpg')}
imageNames = [name for name, image in images.items()]
imagesArray = [np.array(cv2.imread(name)) for name, image in images.items()]
names = []
# for i in imageNames:
#     print(name)
#     names.append(i.split("\\")[-1])
#     print(imageNames)

Done. Reconstruction error: 0.000137464


In [8]:
# Plot result
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image
import plotly.express as px

fig = plt.figure(figsize=(15,15))



# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:,1])

# ax.set_title("Original data")

def getImage(path):
    im = Image.open(path)
#     (width, height) = (im.width // 10, im.height // 10)
    (width, height) = (80, 45)
    im_res = im.resize((width, height))
    return OffsetImage(im_res)


ax = fig.add_subplot(212)
# ax.margins(0.05) 
# plt.figure(figsize=(30,30))
plt.scatter(LLEReduced[:, 0], LLEReduced[:, 1])
# plt.xlim([-0.5, 0.5])
# plt.ylim([-0.5, 0.5])
# for i, txt in enumerate(names):
#     plt.annotate(txt, (LLEReduced[i, 0] + 0.005, LLEReduced[i, 1] + 0.005), fontsize=12)
# for x0, y0, path in zip(annotateX, annotateY, imageNames):
#     ab = AnnotationBbox(getImage(path), (x0,y0), frameon=False)
#     ax.add_artist(ab)
    
for i, path in zip(range(len(LLEReduced)), imageNames):
    ab = AnnotationBbox(getImage(path), (LLEReduced[i, 0], LLEReduced[i, 1]), frameon=False)
    ax.add_artist(ab)
    
ax = fig.add_subplot(211)

# ax.margins(0.05) 
# plt.figure(figsize=(30,30))
plt.scatter(LLEReduced[:, 0], LLEReduced[:, 1])
# plt.xlim([-0.5, 0.5])
# plt.ylim([-0.5, 0.5])
plt.axis('tight')
# plt.xticks([]), plt.yticks([])
plt.title('Projected data')
plt.show()
# annotate

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# K means cluster

In [9]:
from sklearn.cluster import KMeans
cents = np.array([[-0.04767067,  0.40490513],
         [-0.10724884, -0.07074098],
         [ 0.25951771, -0.02275566]])

# kmeans = KMeans(n_clusters = 3, random_state=0, init = cents)
kmeans = KMeans(n_clusters = 3, random_state=0)

#predict the labels of clusters.
label_train = kmeans.fit_predict(LLEReduced)
 
#Getting unique labels
u_labels_train = np.unique(label_train)
centroids = kmeans.cluster_centers_

#plotting the results:
plt.figure(figsize=(15,10))
plt.xlim([-0.5, 0.5])
plt.ylim([-0.5, 0.5])
for i in u_labels_train:
    plt.scatter(LLEReduced[label_train == i , 0] , LLEReduced[label_train == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
centroids

array([[-0.04767067,  0.40490513],
       [-0.10724884, -0.07074098],
       [ 0.25951771, -0.02275566]])

# Testing the KMeans Model for a new image

In [11]:
testimages = {name: cv2.imread(name) for name in glob.glob('./Images\imtestsamples/*jpg')}
LLEVideoArrayTest = []
for name, image in testimages.items():
    plt.figure()
    plt.imshow(image)
    print(name)   
#     cv2.imshow('image',image)
#     cv2.waitKey(0)s
    normalized_X_coordinates, normalized_Y_coordinates = get_ten_joints(image)
    print("normalized_X_coordinates = ", normalized_X_coordinates)
    print("normalized_Y_coordinates = ", normalized_Y_coordinates)
    LLEVideoArrayTest.append([normalized_X_coordinates, normalized_Y_coordinates])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Images\imtestsamples\cs 1.jpg
normalized_X_coordinates =  [-2.1858955782968605, 0.0, 27.500753392631886, -22.600116022830996, 16.28458656118688, -26.40086809074245, -29.70065918665577, -12.019235190849946, 11.43049969909272, -21.388297704394887]
normalized_Y_coordinates =  [-52.46816361634925, 0.0, 28.290430652467826, 22.08839301287272, 134.63307309664037, 116.29678180628416, 112.48251708785882, 2.004064911860598, 178.85547113475616, 173.43097059635454]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Images\imtestsamples\cs 10.jpg
normalized_X_coordinates =  [-2.616034491395573, 0.0, 26.734998955890184, -23.393187485143816, 30.303009267218194, -24.215707992815407, -10.892548315757812, -6.986289508739059, 19.37050696727783, -11.664196495015892]
normalized_Y_coordinates =  [-49.26337974686776, 0.0, 19.733015087343038, 20.594980447523064, 138.53195165434192, 124.75530418657489, 121.46208795808978, 0.305831432342535, 169.99726574267115, 172.27432564381633]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

./Images\imtestsamples\shrug 9.jpg
normalized_X_coordinates =  [-1.2688153935176896, 0.0, 26.442437142402195, -23.757325307474485, 36.76940475229744, -33.40616960514774, 58.781565672105486, -53.27021416511622, 15.634276845433684, -17.25681838285816]
normalized_Y_coordinates =  [-44.07844217618306, 0.0, 17.75452689329783, 19.12959591547648, 100.66103974978128, 110.72475210825601, 54.767520825068154, 42.56051687399546, 168.4165736834208, 164.73985270659128]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Images\imtestsamples\z 1.jpg
normalized_X_coordinates =  [11.09646400436759, 0.0, -32.37149212509394, 17.878925427794456, -70.79292554408312, 16.52287319302559, -101.65360290557146, -5.773990415036678, -70.28472609817982, -37.958589382469654]
normalized_Y_coordinates =  [-23.90841552354757, 0.0, -13.723483450204423, 34.80145216566845, -42.0948944334845, 46.428961748058356, -80.6256084187517, -0.9069473130031697, 136.99835993132544, 155.6401172598589]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Images\imtestsamples\z 2.jpg
normalized_X_coordinates =  [0.2520817397820793, 0.0, 21.581951566416805, -28.431793354978467, 61.012149007633475, -34.466070508716086, 56.10125910754155, -32.3257896334234, 18.403985434108307, -15.346368003373197]
normalized_Y_coordinates =  [-44.07727005377032, 0.0, 13.600576042115858, 26.636778835534376, 32.3188340866532, 120.45814218500855, -75.34985171667915, 192.58413308909527, 164.02168845488734, 168.5962067194093]


In [12]:
TestLLEFrames = []

In [13]:
for k in range(len(LLEVideoArrayTest)):
    temp = []
    for i in range(len(LLEVideoArrayTest[k][0])):
        temp.append(LLEVideoArrayTest[k][0][i])
        temp.append(LLEVideoArrayTest[k][1][i])           
    TestLLEFrames.append(temp)
print(TestLLEFrames)

[[-2.1858955782968605, -52.46816361634925, 0.0, 0.0, 27.500753392631886, 28.290430652467826, -22.600116022830996, 22.08839301287272, 16.28458656118688, 134.63307309664037, -26.40086809074245, 116.29678180628416, -29.70065918665577, 112.48251708785882, -12.019235190849946, 2.004064911860598, 11.43049969909272, 178.85547113475616, -21.388297704394887, 173.43097059635454], [-2.616034491395573, -49.26337974686776, 0.0, 0.0, 26.734998955890184, 19.733015087343038, -23.393187485143816, 20.594980447523064, 30.303009267218194, 138.53195165434192, -24.215707992815407, 124.75530418657489, -10.892548315757812, 121.46208795808978, -6.986289508739059, 0.305831432342535, 19.37050696727783, 169.99726574267115, -11.664196495015892, 172.27432564381633], [-0.16759420361588984, -47.3510909755275, 0.0, 0.0, 27.155466903658475, 27.136585341309594, -23.16767893293325, 19.51170951690313, 26.62040280068622, 122.41655363226835, -20.956713947303157, 98.24008143173073, -26.60160250961781, 91.76032498197735, 0.63

In [14]:
from sklearn import manifold, datasets
LLEReducedTest, err = manifold.locally_linear_embedding(TestLLEFrames, n_neighbors=20, n_components=2)
print("Done. Reconstruction error: %g" % err)

Done. Reconstruction error: 1.10633e-05


In [15]:
print(len(LLEReduced))

33


In [16]:
LLEReduced

array([[-0.0466143 ,  0.38651814],
       [-0.05758882,  0.43953689],
       [-0.02118354,  0.43728638],
       [-0.065296  ,  0.3562791 ],
       [ 0.14786429,  0.02144473],
       [-0.14683095, -0.19898169],
       [ 0.20867862, -0.13235983],
       [ 0.01422757, -0.07315589],
       [-0.14627787, -0.15078361],
       [ 0.27418343, -0.05797149],
       [ 0.24639551, -0.07676665],
       [-0.20853366, -0.06073439],
       [-0.07913967, -0.28143431],
       [ 0.31670965,  0.03974325],
       [ 0.30949045,  0.02632844],
       [ 0.30636824,  0.010272  ],
       [ 0.29774577,  0.00145224],
       [ 0.0258155 , -0.05128603],
       [-0.03454024,  0.12421711],
       [-0.06537179, -0.07545688],
       [ 0.22822345, -0.03694363],
       [-0.13490107, -0.03813375],
       [-0.21552373, -0.12560815],
       [ 0.01365551, -0.26667826],
       [-0.0898231 , -0.11467801],
       [-0.16281442,  0.06509365],
       [-0.00553294, -0.07987792],
       [-0.24825881, -0.05942465],
       [-0.10952862,

In [17]:
print(len(LLEReducedTest))

46


In [18]:
LLEReducedTest

array([[-1.91417810e-01, -8.71444211e-02],
       [-1.72262778e-01, -8.20845640e-02],
       [-1.76364913e-01, -1.02423950e-01],
       [-1.92273283e-01, -8.26894445e-02],
       [-1.50076807e-01, -1.07731334e-01],
       [-1.55959758e-01, -7.93163773e-02],
       [-2.39251554e-01, -9.22385468e-02],
       [-1.70562910e-01, -9.68524803e-02],
       [-1.91617182e-01, -7.75946651e-02],
       [-1.87671154e-01, -9.06648838e-02],
       [-1.52199844e-01, -1.04787058e-01],
       [-1.84967625e-01, -8.88636279e-02],
       [-1.17816376e-01, -1.09771840e-01],
       [-1.82620664e-01, -8.66163514e-02],
       [-2.03557304e-01, -7.97949856e-02],
       [-2.73586827e-02,  2.91935279e-01],
       [-3.45027252e-02,  2.83208132e-01],
       [-4.60921613e-02,  3.01348030e-01],
       [-2.91344578e-02,  2.45861504e-01],
       [-5.75038548e-02,  2.32449567e-01],
       [ 4.33091884e-03,  2.67963940e-01],
       [-4.24035028e-02,  2.65894365e-01],
       [-2.76927157e-04,  2.55174788e-01],
       [-2.

In [19]:
from sklearn.cluster import KMeans

#predict the labels of clusters.
label = kmeans.fit_predict(LLEReducedTest)
 
#Getting unique labels
u_labels = np.unique(label)
centroids = kmeans.cluster_centers_

#plotting the results:
plt.figure(figsize=(15,10))
# plt.gca().invert_yaxis()
for i in u_labels:
    plt.scatter(LLEReducedTest[:][label == i , 0] , LLEReducedTest[:][label == i , 1] , label = i)
plt.scatter(LLEReducedTest[-1][0] , LLEReducedTest[-1][1] ,color = 'red')
plt.scatter(LLEReducedTest[-2][0] , LLEReducedTest[-2][1] ,color = 'red')
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black')
plt.legend(['Chin Scratch','Idle','Shrugging','Outliers'])
plt.xlim([-0.5, 0.5])
plt.ylim([0.5, -0.5])
# plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
print(label)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 0 2]


In [21]:
print(u_labels)

[0 1 2]


In [22]:
dict = {0: 'idle',
        1: 'chin scrathc',
        2: 'shrugging'}
dict_count = {}
for i in range(0,44):
    y_pred = kmeans.predict([LLEReducedTest[-i]])
    print("The test image belongs to the cluster:", dict[y_pred[0]])
    if (dict[y_pred[0]]) in dict_count:
        dict_count[dict[y_pred[0]]] += 1
    else:
        dict_count[dict[y_pred[0]]] = 1

The test image belongs to the cluster: idle
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: idle
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the cluster: shrugging
The test image belongs to the 

In [23]:
print(dict_count)

{'idle': 14, 'shrugging': 20, 'chin scrathc': 10}
